## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов.

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор.

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from google.colab import drive
drive.mount('/content/drive')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/machine_lerning/Lab 5/sms_spam.csv")
df['type'] = df['type'].replace({'ham': 0, 'spam': 1})

df

,type,text
0,0,Hope you are having a good week. Just checking in
1,0,K..give back my thanks.
2,0,Am also doing in cbe only. But have to pay.
3,1,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,1,okmail: Dear Dave this is your final notice to...
...,...,...
5554,0,You are a great role model. You are giving so ...
5555,0,"Awesome, I remember the last time we got someb..."
5556,1,"If you don't, your prize will go to another cu..."
5557,1,"SMS. ac JSco: Energy is high, but u may not kn..."


In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text)    # токенизация
    lemmatizer = WordNetLemmatizer() # применяем лемматизация
    tokens = [token.lower() for token in tokens] # приведение к нижнему регистру
    tokens = [token for token in tokens if token not in ENGLISH_STOP_WORDS]    # удаляем стоп-слова
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    preprocessed_text = ' '.join(tokens)  # склеиваем токены обратно в текстовую строку
    return preprocessed_text

df['text'] = df['text'].apply(preprocess_text)

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['type'], test_size=0.2, random_state=42)

In [79]:
#bag n-gramm
vectorizer_bow = CountVectorizer(ngram_range=(1, 3))
X_train_bow = vectorizer_bow.fit_transform(X_train)
X_test_bow = vectorizer_bow.transform(X_test)
model_bow = MultinomialNB()
model_bow.fit(X_train_bow, y_train)
y_pred_bow = model_bow.predict(X_test_bow)

#tf-idf
vectorizer_tfidf = TfidfVectorizer(ngram_range=(1, 5), max_df=0.99, min_df=0.001, max_features=1000)
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_test_tfidf = vectorizer_tfidf.transform(X_test)
model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

#char n-gramm
vectorizer_char = CountVectorizer(ngram_range=(2, 2), analyzer='char')
X_train_char = vectorizer_char.fit_transform(X_train)
X_test_char = vectorizer_char.transform(X_test)
model_char = MultinomialNB()
model_char.fit(X_train_char, y_train)
y_pred_char = model_char.predict(X_test_char)

In [82]:
metrics = {
    'precision': [precision_score(y_test, y_pred_bow), precision_score(y_test, y_pred_tfidf), precision_score(y_test, y_pred_char)],
    'recall': [recall_score(y_test, y_pred_bow), recall_score(y_test, y_pred_tfidf), recall_score(y_test, y_pred_char)],
    'f1-score': [f1_score(y_test, y_pred_bow), f1_score(y_test, y_pred_tfidf), f1_score(y_test, y_pred_char)],
    'accuracy': [accuracy_score(y_test, y_pred_bow), accuracy_score(y_test, y_pred_tfidf), accuracy_score(y_test, y_pred_char)]
}

df_metrics = pd.DataFrame(metrics, index=['Bag of N-grams', 'TF-IDF', 'Character N-grams'])
df_metrics

,precision,recall,f1-score,accuracy
Bag of N-grams,0.964286,0.906040,0.934256,0.982914
TF-IDF,0.954887,0.852349,0.900709,0.974820
Character N-grams,0.972603,0.953020,0.962712,0.990108


In [76]:
def find_best_model_bow(X_train, X_test, y_train):
    best_score = 0
    best_params = {}

    for ngram_range in [(1, 1), (1, 2), (1, 3)]:
        vectorizer_bow = CountVectorizer(ngram_range=ngram_range)
        X_train_bow = vectorizer_bow.fit_transform(X_train)
        X_test_bow = vectorizer_bow.transform(X_test)
        model_bow = MultinomialNB()
        model_bow.fit(X_train_bow, y_train)
        y_pred_bow = model_bow.predict(X_test_bow)

        score = accuracy_score(y_test, y_pred_bow)

        if score > best_score:
            best_score = score
            best_params['ngram_range'] = ngram_range

    return best_params, best_score

print(find_best_model_bow(X_train, X_test, y_train))

({'ngram_range': (1, 2)}, 0.9856115107913669)


In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

def find_best_model_tfidf(X_train, X_test, y_train, y_test):
    parameters = {
        'ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)],
        'max_df': [0.9, 0.95, 0.99],
        'min_df': [0.0001, 0.001, 0.01],
        'max_features': [500, 1000, 1500]
    }

    vectorizer_tfidf = TfidfVectorizer()
    clf = MultinomialNB()
    grid_search = GridSearchCV(clf, parameters)

    X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
    X_test_tfidf = vectorizer_tfidf.transform(X_test)

    grid_search.fit(X_train_tfidf, y_train)

    best_model = grid_search.best_estimator_
    y_pred_tfidf = best_model.predict(X_test_tfidf)

    return best_model, y_pred_tfidf


print(find_best_model_tfidf(X_train, X_test, y_train, y_test))

({'tf-idf': (1, 5)}, 0.973022)


In [81]:
def find_best_model_char(X_train, X_test, y_train):
    best_score = 0
    best_params = {}

    for n in range(2, 6):
        for analyzer in ['char', 'char_wb']:
            vectorizer = CountVectorizer(ngram_range=(n, n), analyzer=analyzer)
            X_train_vec = vectorizer.fit_transform(X_train)
            X_test_vec = vectorizer.transform(X_test)
            model = MultinomialNB()
            model.fit(X_train_vec, y_train)
            y_pred = model.predict(X_test_vec)
            score = accuracy_score(y_test, y_pred)

            if score > best_score:
                best_score = score
                best_params['ngram_range'] = (n, n)
                best_params['analyzer'] = analyzer

    return best_params, best_score

print(find_best_model_char(X_train, X_test, y_train))

({'ngram_range': (2, 2), 'analyzer': 'char'}, 0.9901079136690647)


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах.

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **а?** - ноль или один символ **а**
* **а+** - один или более символов **а**
* **а\*** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа


In [ ]:
result = re.findall('a?b.', 'aabbсabbcbb')
print(result)

['abb', 'abb', 'bb']


In [ ]:
result = re.findall('a*b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb', 'bb']


In [ ]:
result = re.findall('a+b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb']


Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**:
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc')
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
result = re.findall(r'\b[a-zA-Z]{2}', 'they have no idea') #дефисы
result

['th', 'ha', 'no', 'id']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie')
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2)
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
result = re.split('\.', 'i have mo idea. i have no idea. i have no idea.', maxsplit=2)
result

['i have mo idea', ' i have no idea', ' i have no idea.']

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
result = re.sub('(?<!\d)\d(?!\d)', 'DIG', 'itmo3 it mo re 45 45 67')
print(result)

itmoDIG it mo re 45 45 67


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
result = re.sub('((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', '', 'https://piazza.com/class/llzhanymd06me/post/105 456545')
result

' 456545'

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile('[\w-]{4,}')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'больше', 'больше', 'слов', 'Что-то']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
prog = re.compile(r'[@]\w+[.]\w{2,6}')
prog.findall("abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz")

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']